**ძირითადი ნაწილი (6 ქულა)**

ამ დავალების პირობა თქვენთვის კარგად ნაცნობია შუალედური გამოცდიდან. 😁😁 მოგიწევთ, რომ იდენტური ამოცანა ამოხსნათ, ოღონდ PySpark-ის გამოყენებით. არ გამოიყენოთ Pandas-ის ბიბლიოთეკა. წინააღმდეგ შემთხვევაში არ ჩაითვლება. 

შეგახსენებთ პირობას:

You are given dataset of Marvel superheroes (some DC heroes may have sneaked through as usual), your first task is to randomly generate teams of 5 heroes with following columns:
- team_name: team name should be generated using strongest hero name in team with following template "Team: {Strongest_Hero_Name}" to calculate stongest hero in team you'll need to calculate average of following attributes in charcters_stats file: Intelligence, Strength, Speed, Durability, Power, Combat
- leader: name of hero with highest Intelligence score in team
- tank: name of hero with highest sum value of Strength and Durability
- damage: hero with highest sum value of Speed, Power and Combat 
- the_other_guy: one of the hero who's not specified in above columns
- the_other_guy2: one of the hero who's not specified in above columns
- top_average_speed: average of top 3 highest speed entries in team
- top_average_height: average of top 3 tallest heroes in team
- flight_count: number of heroes who can fly in team (available in superheroes_power_matrix)  

**same hero should not end up in multiple teams.**

**all negative values should be converted to 0.**


თუმცა, შუალედურისგან განსხვავებით ერთი შეზღუდვაა: *ამოხსნაში არ უნდა გამოიყენოთ ციკლები (for, while და მისთანანი   
🥲)*.

In [ ]:
# ძირითადი ნაწილი spark 3.2-ზე
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, count, when
from pyspark.sql.types import *

app_name = 'DataEngineering'

spark = SparkSession \
      .builder \
      .enableHiveSupport() \
      .config("hive.exec.dynamic.partition", "true") \
      .config("hive.exec.dynamic.partition.mode", "nonstrict") \
      .appName(app_name) \
      .getOrCreate()\


sc = spark.sparkContext

In [ ]:
character_stats = spark.read.option('inferSchema', 'true') \
                  .option('header', 'true') \
                  .csv('character_stats.csv')

marvel_characters_info = spark.read.option('inferSchema', 'true') \
                         .option('header', 'true') \
                         .csv('marvel_characters_info.csv')

superheroes_power_matrix = spark.read.option('inferSchema', 'true') \
                           .option('header', 'true') \
                           .csv('superheroes_power_matrix.csv')

chars_not_in_character_stats1 = marvel_characters_info.join(character_stats, character_stats.Name == marvel_characters_info.Name, 'leftanti')
chars_not_in_character_stats2 = superheroes_power_matrix.join(character_stats, character_stats.Name == superheroes_power_matrix.Name, 'leftanti')
chars_not_in_character_stats = chars_not_in_character_stats1.select(col('Name')).union(chars_not_in_character_stats2.select(col('Name'))).distinct()

chars_not_in_mci1 = character_stats.join(marvel_characters_info, marvel_characters_info.Name == character_stats.Name, 'leftanti')
chars_not_in_mci2 = superheroes_power_matrix.join(marvel_characters_info, marvel_characters_info.Name == superheroes_power_matrix.Name, 'leftanti')
chars_not_in_mci = chars_not_in_mci1.select(col('Name')).union(chars_not_in_mci2.select(col('Name'))).distinct()

chars_not_in_spm1 = character_stats.join(superheroes_power_matrix, character_stats.Name == superheroes_power_matrix.Name, 'leftanti')
chars_not_in_spm2 = marvel_characters_info.join(superheroes_power_matrix, marvel_characters_info.Name == superheroes_power_matrix.Name, 'leftanti')
chars_not_in_spm = chars_not_in_spm1.select(col('Name')).union(chars_not_in_spm2.select(col('Name'))).distinct()

character_stats = character_stats.unionByName(chars_not_in_character_stats, allowMissingColumns=True).sort(col('Name'))
character_stats = character_stats.fillna(0, subset=['Intelligence', 'Strength', 'Speed', 'Durability', 'Power', 'Combat', 'Total'])

marvel_characters_info = marvel_characters_info.unionByName(chars_not_in_mci, allowMissingColumns=True).sort(col('Name'))
marvel_characters_info = marvel_characters_info.fillna(0, subset=['Height', 'Weight'])

superheroes_power_matrix = superheroes_power_matrix.unionByName(chars_not_in_spm, allowMissingColumns=True).sort(col('Name'))
superheroes_power_matrix = superheroes_power_matrix.fillna(False)

marvel_characters_info = marvel_characters_info.withColumn('Height', when(col("Height") > 0, col("Height")).otherwise(0))
marvel_characters_info = marvel_characters_info.withColumn('Weight', when(col("Weight") > 0, col("Weight")).otherwise(0))

In [ ]:
# ციკლების გარეშე გადავწყვიტე 8 გუნდი ხელით ჩამომეწერა

sample_chars = character_stats.sample(withReplacement=False, fraction=0.5).limit(40).cache()

team_a = sample_chars.limit(5).cache()

sample_chars = sample_chars.join(team_a, 'Name', 'leftanti')

team_b = sample_chars.limit(5).cache()

sample_chars = sample_chars.join(team_b, 'Name', 'leftanti')

team_c = sample_chars.limit(5).cache()

sample_chars = sample_chars.join(team_c, 'Name', 'leftanti')

team_d = sample_chars.limit(5).cache()

sample_chars = sample_chars.join(team_d, 'Name', 'leftanti')

team_e = sample_chars.limit(5).cache()

sample_chars = sample_chars.join(team_e, 'Name', 'leftanti')

team_f = sample_chars.limit(5).cache()

sample_chars = sample_chars.join(team_f, 'Name', 'leftanti')

team_g = sample_chars.limit(5).cache()

sample_chars = sample_chars.join(team_g, 'Name', 'leftanti')

team_h = sample_chars.cache()

team_a.show()

+-------------+---------+------------+--------+-----+----------+-----+------+-----+
|         Name|Alignment|Intelligence|Strength|Speed|Durability|Power|Combat|Total|
+-------------+---------+------------+--------+-----+----------+-----+------+-----+
|   Abe Sapien|     good|          88|      14|   35|        42|   35|    85|  299|
|      Abraxas|      bad|          88|     100|   83|        99|  100|    56|  526|
|Absorbing Man|     null|           0|       0|    0|         0|    0|     0|    0|
| Adam Strange|     good|           1|       1|    1|         1|    0|     1|    5|
|   Agent Zero|     good|          75|      28|   38|        80|   72|    95|  388|
+-------------+---------+------------+--------+-----+----------+-----+------+-----+



In [ ]:
def make_team(team_original):
    team = team_original.alias('team').cache()
    team_name = 'Team: ' + team.select(col('Name'), ((col('Intelligence') + col('Strength') + col('Speed') \
                                                      + col('Durability') + col('Power') + col('Combat')) / 6) \
                                                      .alias('avg')).sort(col('avg'), ascending=False).first()[0]
    team_leader = team.select(col('Name'), col('Intelligence')).sort(col('Intelligence'), ascending=False).first()[0]
    team = team.filter(col('Name') != team_leader)
    team_tank = team.select(col('Name'), (col('Strength') + col('Durability')).alias('Tankiness')) \
                    .sort(col('Tankiness'), ascending=False).first()[0]
    team = team.filter(col('Name') != team_tank)
    team_damage = team.select(col('Name'), (col('Speed') + col('Power') + col('Combat')).alias('Damage')) \
                      .sort(col('Damage'), ascending=False).first()[0]
    team = team.filter(col('Name') != team_damage)
    team_other_guy = team.first()[0]
    team = team.filter(col('Name') != team_other_guy)
    team_other_guy2 = team.first()[0]
    heights = team_original.join(marvel_characters_info, marvel_characters_info.Name == team_original.Name, 'inner') \
                           .select(col('Height')) \
                           .sort(col('Height'), ascending=False).take(3)
    top_average_height = (heights[0][0] + heights[1][0] + heights[2][0]) / 3
    speeds = team_original.select(col('Speed')).sort(col('Speed'), ascending=False).take(3)
    top_average_speed = (speeds[0][0] + speeds[1][0] + speeds[2][0]) / 3
    flight = team_original.join(superheroes_power_matrix, superheroes_power_matrix.Name == team_original.Name, 'inner') \
                          .select(col('Flight'))
    flight_count = flight.filter(col('Flight') == 'true').count()
    return (team_name, team_leader, team_tank, team_damage, team_other_guy, team_other_guy2, top_average_speed,
            top_average_height, flight_count)

cols = ['team_name', 'leader', 'tank', 'damage', 'the_other_guy', 'the_other_guy2', 'top_average_speed', 'top_average_height', 'flight_count']

# 1 წუთამდე სჭირდება ამ ნაწილს
team_a = make_team(team_a)
team_b = make_team(team_b)
team_c = make_team(team_c)
team_d = make_team(team_d)
team_e = make_team(team_e)
team_f = make_team(team_f)
team_g = make_team(team_g)
team_h = make_team(team_h)
team_a


('Team: Abraxas',
 'Abe Sapien',
 'Abraxas',
 'Agent Zero',
 'Absorbing Man',
 'Adam Strange',
 52.0,
 191.66666666666666,
 2)

In [ ]:
import pandas as pd
teams = [team_a, team_b, team_c, team_d, team_e, team_f, team_g, team_h]
# პირველი ნაწილი 3.2-ზე დაწყებული მქონდა უკვე და სამწუხაროდ map და სხვა ფუნქციონალები არ მუშაობდა
# teams_df = spark.createDataFrame(teams, cols) ერორს მიგდებდა ამის გამოტანისას spark 3+ზე და პანდასით უბრალოდ დავაექსპორტე
pd_df = pd.DataFrame(teams, columns=cols)
pd_df.set_index('team_name').to_csv('./df.csv')

In [2]:
teams_df = spark.read.option('inferSchema', 'true') \
           .option('header', 'true') \
           .csv('df.csv')
teams_df.show()

+----------------+----------+------------+----------------+--------------+--------------+------------------+------------------+------------+
|       team_name|    leader|        tank|          damage| the_other_guy|the_other_guy2| top_average_speed|top_average_height|flight_count|
+----------------+----------+------------+----------------+--------------+--------------+------------------+------------------+------------+
|   Team: Abraxas|Abe Sapien|     Abraxas|      Agent Zero| Absorbing Man|  Adam Strange|              52.0|191.66666666666666|           2|
|Team: Air-Walker|Air-Walker|Alex Woolsly|Allan Quatermain|          Ajax|   Alex Mercer|              34.0|             127.0|           1|
|Team: Animal Man|   Ant-Man|  Animal Man|            Ammo|Ando Masahashi|    Anti-Spawn|23.666666666666668|             194.0|           0|
|Team: Apocalypse|Apocalypse|     Aquaman|      Anti-Venom|      Aquababy|      Arclight|49.333333333333336|             209.0|           1|
|     Team: A

**ბონუს ნაწილი (2 ქულა)**

ეს ნაწილიც თქვენთვის ნაცნობია და გუნდების ერთმანეთთან შეჯიბრებას, ბოლოს კი გამარჯვებულის გამოვლენას ეხება.
შეგახსენებთ პირობას:

After generating teams, obvious next task would be to perform basketball face off between teams. To determine winning teams we'll be using following logic:
* team scores 1 point if their top_average_speed is superior 
* team scores 1 point if their top_average_height is superior 
* team scores 1 point for each member inside team who can fly (available in superheroes_power_matrix)  
team with highest score wins!

Based on rules above you should perform competition: first you'll need to randomly divide teams (all of them) into 4 brackets and generate data frames for each of them, data frame should contain all possible combinations of teams,
for example if bracket consists of teams [A,B,C,D] data frame should contain pairs (A,B) (A,C) (A,D) (B,C) (B,D) (C,D) with following schema
- team_a: name for first team
- team_b: name for second team
- team_a_score: score for first team against second team, calculated using rules described above
- team_b_score: same for second team  

after generating data frame, pick one team from each bracket with most wins: team1, team2, team3 and team4
perform semi-finals: team1 vs team2, team3 vs team4 and finals with winner from these games

**if you get a tie at any point when facing teams with each other, you can pick any team, you may also drop some teams if there is too much missing data**  

---

გაწვდით 3 ფაილს, რომელიც ამ დავალებისთვის დაგჭირდებათ. ფაილებში პერსონაჟები არ დუბლირდება და შეგიძლიათ, ეგ არ შეამოწმოთ.

ამოხსნა ატვირთეთ ნოუთბუქის სახით.

წარმატებები!

## ბონუს ნაწილი spark 2.4-ზე გავაკეთე ქოლაბით

In [3]:
from random import random

teams_df_copy = teams_df.alias('copy')

teams_in_bracket = int(teams_df.count() / 4)
# randomSplit ვცადე მაგრამ არაკონსისტენტური იყო, ზოგ ჯგუფში 1-ით მეტი ან ნაკლები ხვდებოდა
teams_df = teams_df.sample(False, 1.0, random()) # რენდომად უნდა არიოს წესით სტრიქონები
first_bracket = teams_df.limit(teams_in_bracket).cache()
teams_df = teams_df.join(first_bracket, 'team_name', 'leftanti')
second_bracket = teams_df.limit(teams_in_bracket).cache()
teams_df = teams_df.join(second_bracket, 'team_name', 'leftanti')
third_bracket = teams_df.limit(teams_in_bracket).cache()
teams_df = teams_df.join(third_bracket, 'team_name', 'leftanti')
fourth_bracket = teams_df.cache()
first_bracket.show()

+----------------+----------+------------+----------------+-------------+--------------+-----------------+------------------+------------+
|       team_name|    leader|        tank|          damage|the_other_guy|the_other_guy2|top_average_speed|top_average_height|flight_count|
+----------------+----------+------------+----------------+-------------+--------------+-----------------+------------------+------------+
|   Team: Abraxas|Abe Sapien|     Abraxas|      Agent Zero|Absorbing Man|  Adam Strange|             52.0|191.66666666666666|           2|
|Team: Air-Walker|Air-Walker|Alex Woolsly|Allan Quatermain|         Ajax|   Alex Mercer|             34.0|             127.0|           1|
+----------------+----------+------------+----------------+-------------+--------------+-----------------+------------------+------------+



In [4]:
from itertools import combinations
def getWinner(bracket):
  team_names = bracket.rdd.map(lambda x: x[0]).collect()
  team_combinations = list(combinations(team_names, 2))
  combinations_df = spark.createDataFrame(team_combinations, ['team_a', 'team_b'])
  comb_with_scores = combinations_df.join(bracket.select(col('team_name').alias('team_a_name'),
                                        col('top_average_speed').alias('team_a_speed'), 
                                        col('top_average_height').alias('team_a_height'), 
                                        col('flight_count').alias('team_a_flight')), 
                                        col('team_a_name') == col('team_a'), 'inner') \
                                    .join(bracket.select(col('team_name').alias('team_b_name'),
                                        col('top_average_speed').alias('team_b_speed'), 
                                        col('top_average_height').alias('team_b_height'), 
                                        col('flight_count').alias('team_b_flight')), 
                                        col('team_b_name') == col('team_b'), 'inner') \
                                    .rdd.map(lambda row: (row[0], row[1], (1 if row[3] > row[7] else 0)
                                        + (1 if row[4] > row[8] else 0) + row[5],
                                        (1 if row[7] > row[3] else 0) + (1 if row[8] > row[4]
                                        else 0) + row[9])).collect()
  score_df = spark.createDataFrame(comb_with_scores, ['team_a', 'team_b', 'team_a_score', 'team_b_score'])
  bracket_winner = score_df.rdd.map(lambda row: (row[0] if row[2] >= row[3] else row[1], 1)) \
                           .reduceByKey(lambda x, y: x + y) \
                           .sortBy(lambda x: -x[1]).collect()[0][0]
  return bracket_winner

In [5]:
round_one_winners = [getWinner(first_bracket), getWinner(second_bracket), 
                     getWinner(third_bracket), getWinner(fourth_bracket)]
print('Round 1 winners:', round_one_winners)

round_one_winner_1 = teams_df_copy.filter(col('team_name') == round_one_winners[0])
round_one_winner_2 = teams_df_copy.filter(col('team_name') == round_one_winners[1])
round_one_winner_3 = teams_df_copy.filter(col('team_name') == round_one_winners[2])
round_one_winner_4 = teams_df_copy.filter(col('team_name') == round_one_winners[3])

round_two_first_bracket = round_one_winner_1.union(round_one_winner_2)
round_two_second_bracket = round_one_winner_3.union(round_one_winner_4)

round_two_winners = [getWinner(round_two_first_bracket), getWinner(round_two_second_bracket)]
print('Round 2 winners:', round_two_winners)

round_two_winner_1 = teams_df_copy.filter(col('team_name') == round_two_winners[0])
round_two_winner_2 = teams_df_copy.filter(col('team_name') == round_two_winners[1])

final_bracket = round_two_winner_1.union(round_two_winner_2)

finals_winner = getWinner(final_bracket)
print('Finals winner:', finals_winner)

Round 1 winners: ['Team: Abraxas', 'Team: Apocalypse', 'Team: Atlas', 'Team: Beast']
Round 2 winners: ['Team: Abraxas', 'Team: Beast']
Finals winner: Team: Abraxas
